# Republican Caucus Miscellaneous Measurements

This notebook contains some miscellaneous calculations added to our paper during the review process. Among other things, it calculates the Holding Times found in Table 5.

To run this notebook you'll first need to run the Trader Analysis notebooks first.

In [ ]:
import os
import sys
import pickle

from collections import defaultdict

import numpy as np
import pandas as pd
from pandas import Series

from research_tools import storage

# Load Data

In [ ]:
os.chdir('..')

basename = 'gop'

trader_analysis, = storage.retrieve_all([basename + '.trader_analysis'])

In [ ]:
trader_analysis.head()

# Traders with Positions at Market Close

How many traders had a position at market close?

In [ ]:
close_traders = trader_analysis.groupby('user_guid').apply(lambda x: x.close_trade.any())

close_traders.head()

In [ ]:
close_traders.sum() / close_traders.count()

In [ ]:
close_traders.sum(), close_traders.count()

# Average Holding Time

What is the average holding time of positions? And median holding time?

In [ ]:
holding_times = []
position_open_timestamps = defaultdict(list)

for trade in trader_analysis.itertuples():
    if trade.buy_sell == 1:
        position_open_timestamps[(trade.contract_id, trade.user_guid)].extend([trade.seq] * trade.quantity)
    elif trade.buy_sell == -1:
        oldest_timestamps = position_open_timestamps[(trade.contract_id, trade.user_guid)][:trade.quantity]
        for t in oldest_timestamps:
            holding_times.append(trade.seq - t)
        position_open_timestamps[(trade.contract_id, trade.user_guid)] = (
            position_open_timestamps[(trade.contract_id, trade.user_guid)][trade.quantity:]
        )

holding_times = Series(holding_times)

In [ ]:
pd.Timedelta(holding_times.mean(), 'ms')

In [ ]:
holding_times.mean() / (24 * 60 * 60 * 1000)

In [ ]:
pd.Timedelta(holding_times.quantile(0.5), 'ms')

In [ ]:
holding_times.quantile(0.5) / (24 * 60 * 60 * 1000)

Save the holding times data to a pickle file so we can combine it with the data from the other market.

In [ ]:
with open('data/gop.holding_times.p', 'wb') as f:
    pickle.dump(holding_times, f)

# Pre-Margin Linking Volume

How much volume was during the pre-margin linking period?

In [ ]:
rep_margin_linking_cutoff_date = '2015-12-01'
pre_margin_linking = trader_analysis.date_executed < rep_margin_linking_cutoff_date

trader_analysis[pre_margin_linking].quantity.sum() / trader_analysis.quantity.sum()